In [2]:
import numpy as np
import pandas as pd
import imageio
from scipy import misc
from scipy import signal
from skimage import data
from skimage import feature
from skimage import filters
from skimage import io, color
from skimage import morphology
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from scipy import ndimage
from numpy import asarray
from numpy import savez_compressed
from numpy import load
from enum import Enum

<h1 align="left" style="font-size:20px"><b> General Methods </b></h1>

In [ ]:
class SmoothMode(Enum):
    GAUSSIANBLUR="GaussianBlur"
    DECAY="Decay"
    CLIPWEAK="CipWeak"


In [ ]:
def smooth_img(image, sigma_est = None, mode = None):
    #mode checking
    if not isinstance(mode, SmoothMode):
        raise TypeError('mode must be an instance of SmoothMode Enum')
    if mode is None:
        return image
    elif mode == SmoothMode.GAUSSIANBLUR:
        # Gaussian Blurring with width of 3
        if sigma_est is not None:
            img_filtered = ndimage.filters.gaussian_filter(image,sigma_est)
        else:
            img_filtered = ndimage.filters.gaussian_filter(image,1/2)
        return img_filtered
    elif mode == SmoothMode.DECAY:
        # Decay regularization
        decay = 0.98
        img_filtered = decay * image
        return img_filtered
    elif mode == SmoothMode.CLIPWEAK:
        # Clip weak pixel regularization
        percentile = 1
        threshold = np.percentile(np.abs(image),percentile)
        image[np.where(np.abs(image) < threshold)] = 0
        #print(image[np.where(np.abs(image) < threshold)])
        img_filtered = image.copy()
        return img_filtered
    else:
        # print error message
        raise TypeError('Unknown smooth_img parameter. No smooth_img implemented.')
        #print('Unknown smoothing parameter. No smoothing implemented.')
        return image 

In [ ]:
def compare_two_arrays(array_1, array_2):
    return np.array_equal(array_1,array_2)

In [ ]:
class ReadyMadeKernel(Enum):
    SHARPEN="Sharpen"
    BLUR="Blur"
    EDGEENHANCED="EdgeEnhanced"
    EDGEDETECT="EdgeDetec"
    EMBOSS="Emboss"
    GAUSSIANSTATIC="GaussianStatic"
    GAUSSIANAXIS="GaussianAxis"
    GAUSSIANSIGMA="GaussianSigma"

In [4]:
def ReadyMadeKernel_img_RGB(image, sigma_est = None, ready_made_kernel = None):
    #mode checking
    if not isinstance(ready_made_kernel, ReadyMadeKernel):
        raise TypeError('ready_made_kernel must be an instance of ReadyMadeKernel Enum')
    if ready_made_kernel is None:
        return image
    elif ready_made_kernel == ReadyMadeKernel.SHARPEN:
        #Sharpening = edge detection + original image
        #Consider this filter in 1D:
        #[-1 1 0]
        #edge on the left and edge on the right:
        #[-1 1 0] + [0 1 -1] = [-1 2 -1]
        #edge image:
        #[-1 2 -1] + [0 1 0] = [-1 3 -1]
        kernel = np.array([[0.,-1.,0.],
                           [-1.,5.,-1.],
                           [0.,-1.,0.]])
        
        #kernel = np.array([[0, 0, 0, 0, 0], 
        #                   [0, 0, 1, 0, 0], 
        #                   [0, 1, 5, 1, 0],
        #                   [0, 0, 1, 0, 0],
        #                   [0, 0, 0, 0, 0]])
        
        #kernel = kernel[:, :, np.newaxis]
        #img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        
        
        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.BLUR:
        kernel = np.array([[1/16,0.125,1/16],
                        [1/8,1/4,1/8],
                        [1/16,0.125,1/16]])
        
        #kernel = kernel[:, :, np.newaxis]
        img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.GAUSSIANAXIS:
        
        x_dim = image[:,:,0]
        y_dim = image[:,:,1]
        z_dim = image[:,:,2]
        
        if sigma_est is not None:
            image_norm = image.copy()
            image_norm = image_norm / 255
            
            t = 1 - np.abs(np.linspace(-1, 1, 10))
            kernel_sigma = t.reshape(10, 1) * t.reshape(1, 10)
            kernel_sigma /= kernel_sigma.sum()   # kernel should sum to 1!  :)
            
            # convolve 2d the kernel with each channel
            r = signal.convolve2d(image_norm[:,:,0], kernel_sigma, mode='same')
            g = signal.convolve2d(image_norm[:,:,1], kernel_sigma, mode='same')
            b = signal.convolve2d(image_norm[:,:,2], kernel_sigma, mode='same')

            # stack the channels back into a 8-bit colour depth image and plot it
            img_filtered = np.dstack([r, g, b])
            img_filtered = (img_filtered * 255).astype(np.uint8) 

        #print("shape of image: {}".format(kernel_sigma.shape))
        #print("dim of image: {}".format(kernel_sigma.ndim))

        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.GAUSSIANSIGMA:
        xx = np.array([0, 0, 1, 0, 0])
        yy = np.array([0, 0, 1, 0, 0])
        zz = np.array([0, 0, 1, 0, 0])
        
        if sigma_est is not None:
            kx, ky, kz = np.meshgrid(xx,yy,zz)
            kernel_sigma = np.exp(-(kx**2 + ky**2 + kz**2)/(2*sigma_est**2))
            img_filtered = ndimage.convolve(image, kernel_sigma, mode='mirror', cval=0.0)
        #print("shape of image: {}".format(kernel_sigma.shape))
        #print("dim of image: {}".format(kernel_sigma.ndim))

        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.EDGEENHANCED:
        kernel = np.array([ [0, 0, 0, 0, 0], 
                                   [0, 0, -1, 0, 0], 
                                   [0, -1, 5, -1, 0],
                                   [0, 0, -1, 0, 0],
                                   [0, 0, 0, 0, 0]])
        #kernel = kernel[:, :, np.newaxis]
        img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.EDGEDETECT:
        kernel = np.array([ [0, 0, 0, 0, 0], 
                                   [0, 0, 1, 0, 0], 
                                   [0, 1, -4, 1, 0],
                                   [0, 0, 1, 0, 0],
                                   [0, 0, 0, 0, 0]])
        #kernel = kernel[:, :, np.newaxis]
        img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        return img_filtered
    elif ready_made_kernel == ReadyMadeKernel.EMBOSS:
        kernel = np.array([ [0, 0, 0, 0, 0], 
                                   [0, -2, -1, 0, 0], 
                                   [0, -1, 1, 1, 0],
                                   [0, 0, 1, 2, 0],
                                   [0, 0, 0, 0, 0]])
        #kernel = kernel[:, :, np.newaxis]
        img_filtered = ndimage.convolve(image, kernel, mode='mirror', cval=0.0)
        return img_filtered
    else:
        # print error message
        raise TypeError('Unknown ready_made_kernel parameter. No ReadyMadeKernel implemented.')
        #print('Unknown smoothing parameter. No smoothing implemented.')
        return image 

In [ ]:
def bin_gray_to_uint(image):
    return np.uint8(color.rgb2gray(image) * 255)